In [ ]:
import math

# ------------------ ENTROPY FUNCTION ------------------
def entropy(data, target_attr):
    label_count = {}
    
    # Count frequency of each class (Yes/No)
    for row in data:
        label = row[target_attr]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1

    # Calculate entropy
    total = len(data)
    ent = 0
    for label in label_count:
        p = label_count[label] / total
        ent -= p * math.log2(p)
    return ent


# ------------------ INFORMATION GAIN ------------------
def info_gain(data, feature, target_attr):
    total_entropy = entropy(data, target_attr)
    
    # Find unique values of the feature
    values = set([row[feature] for row in data])
    feature_entropy = 0

    for val in values:
        subset = []
        for row in data:
            if row[feature] == val:
                subset.append(row)

        weight = len(subset) / len(data)
        feature_entropy += weight * entropy(subset, target_attr)

    return total_entropy - feature_entropy


# ------------------ ID3 ALGORITHM ------------------
def id3(data, features, target_attr):

    # If all target values are same → return that
    classes = [row[target_attr] for row in data]
    if classes.count(classes[0]) == len(classes):
        return classes[0]

    # If no features left → return majority
    if len(features) == 0:
        return max(classes, key=classes.count)

    # Find best feature using info gain
    gains = {}
    for f in features:
        gains[f] = info_gain(data, f, target_attr)

    best_feature = max(gains, key=gains.get)
    tree = {best_feature: {}}

    # Branch for each value of best feature
    values = set([row[best_feature] for row in data])

    for val in values:
        subset = []
        for row in data:
            if row[best_feature] == val:
                subset.append(row)

        if len(subset) == 0:
            tree[best_feature][val] = max(classes, key=classes.count)
        else:
            new_features = [f for f in features if f != best_feature]
            tree[best_feature][val] = id3(subset, new_features, target_attr)

    return tree


# ------------------ DATASET ------------------
dataset = [
    {"Outlook": "Sunny", "Temp": "Hot",  "Humidity": "High",   "Wind": "Weak",   "Play": "No"},
    {"Outlook": "Sunny", "Temp": "Hot",  "Humidity": "High",   "Wind": "Strong", "Play": "No"},
    {"Outlook": "Overcast", "Temp": "Hot","Humidity": "High", "Wind": "Weak",   "Play": "Yes"},
    {"Outlook": "Rain", "Temp": "Mild",  "Humidity": "High",   "Wind": "Weak",   "Play": "Yes"},
    {"Outlook": "Rain", "Temp": "Cool",  "Humidity": "Normal", "Wind": "Weak",   "Play": "Yes"},
    {"Outlook": "Rain", "Temp": "Cool",  "Humidity": "Normal", "Wind": "Strong", "Play": "No"},
    {"Outlook": "Overcast", "Temp": "Cool","Humidity": "Normal","Wind": "Strong","Play": "Yes"},
    {"Outlook": "Sunny", "Temp": "Mild", "Humidity": "High",   "Wind": "Weak",   "Play": "No"}
]

features = ["Outlook", "Temp", "Humidity", "Wind"]
target = "Play"

tree = id3(dataset, features, target)

print("Decision Tree:")
print(tree)
